
# Катастрофическое забывание

## Цель: Проверить влияние fine-tuning на исходную модель.

## Описание/Пошаговая инструкция выполнения домашнего задания:
1. Скачать датасет ImageNette: https://github.com/fastai/imagenette (`ImageNette` это подвыборка из 10 классов датасета `ImageNet`).
2. Взять предобученную на обычном `ImageNet` модель `ResNet18` и заменить число классов на 10.
3. Дообучить модель на 10 классах `ImageNette` и замерить точность (эта точность будет считаться базовой). Обучить только последний слой. Сохранить последний слой как оригинальный.
4. Дообучить модель классифицировать датасет `CIFAR10`.
5. Вернуть оригинальный последний слой модели и проверить качество на `ImageNette` и сравнить с базовой точностью.
6. Дообучить только последний слой (отключить градиент для всех слоев кроме последнего) на `ImageNette` и проверить удалось ли добиться исходного качества.
7. Сделать выводы.

### Критерии оценки:
__Принято__ - задание выполнено полностью.

__Возвращено на доработку__ - задание не выполнено полностью.


In [159]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

## Шаг 1 Подготовка датасета

### 1.1 Определение настроек

In [189]:
TRAIN_DIR_IMAGENETTE = 'DataForModel/imagenette2-320/train'
VAL_DIR_IMAGENETTE = 'DataForModel/imagenette2-320/val'
TRAIN_DIR_CIFAR10 = 'DataForModel/cifar10/train'
VAL_DIR_CIFAR10 = 'DataForModel/cifar10/val'
BATCH_SIZE = 32
NUM_EPOCHS = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

### 1.2 Датасет ImageNette

In [190]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset_imagenette = torchvision.datasets.ImageFolder(TRAIN_DIR_IMAGENETTE, transform=transform)
val_dataset_imagenette = torchvision.datasets.ImageFolder(VAL_DIR_IMAGENETTE, transform=transform)

imagenette_train = torch.utils.data.DataLoader(train_dataset_imagenette, batch_size=BATCH_SIZE, shuffle=True)
imagenette_val = torch.utils.data.DataLoader(val_dataset_imagenette, batch_size=BATCH_SIZE)

### 1.3 Датасет CIFAR10

In [191]:
train_dataset_cifar = torchvision.datasets.CIFAR10(root=TRAIN_DIR_CIFAR10, train=True, download=True, transform=transform)
val_dataset_cifar = torchvision.datasets.CIFAR10(root=VAL_DIR_CIFAR10, train=False, download=True, transform=transform)

cifar_train = torch.utils.data.DataLoader(train_dataset_cifar, BATCH_SIZE, shuffle=True)
cifar_val = torch.utils.data.DataLoader(val_dataset_cifar, BATCH_SIZE)

### 1.4 Определение фунций обучения и расчета точности модели

In [192]:
# Расчет точности
def evaluate(model, val_loader):
    model.eval() 
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def model_train(model, train_data, valid_data, optimizer, name='undefined', save_fc=False):
    criterion = nn.CrossEntropyLoss()
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        last_acc = 0.0
        model.train()
        for inputs, labels in tqdm(train_data, desc=f'Epoch {epoch+1}'):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Валидация
        acc = evaluate(model, valid_data)
        if acc > last_acc:
            last_acc = acc
            if save_fc:
                torch.save(model.fc.state_dict(), f'{name}.pth')
    return model, last_acc 

## Шаг 2: Замена последнего слоя на 10 классов

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Заменяем последний слой на 10 классов
model.fc = nn.Linear(model.fc.in_features, 10)
# Разморозка последнего слоя
for param in model.fc.parameters():
    param.requires_grad = True
model = model.to(DEVICE)

c:\Users\KharitWinPC\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KharitWinPC\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Шаг 3: Обучение на ImageNette (последний слой)

### Шаг 3.1 Обучение модели

In [197]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.0001)
base_acc = 0.0

model, base_acc = model_train(model, imagenette_train, imagenette_val, optimizer,'imagenette_fc', True)
print(f"Базовая точность: {base_acc:.2f}%")

Epoch 5: 100%|██████████| 296/296 [03:44<00:00,  1.32it/s]


Базовая точность: 97.15%


## Шаг 4: Обучение на CIFAR10 (вся модель)

In [199]:
# Разморозка
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
cifar_acc = 0.0

model, cifar_acc = model_train(model, cifar_train, cifar_val,  optimizer, save_fc=False)
print(f"CIFAR точность: {cifar_acc:.2f}%")

Epoch 5: 100%|██████████| 1563/1563 [49:11<00:00,  1.89s/it]


CIFAR точность: 93.68%


## Шаг 5: Возврат оригинального слоя и проверка качества на ImageNette

### 5.1 Возврат оригинального слоя

In [200]:
model.fc.load_state_dict(torch.load('imagenette_fc.pth'))

<All keys matched successfully>

### 5.2 Проверка качестка на ImageNette

In [201]:
new_acc = evaluate(model, imagenette_val)
print(f"Базовая точность: {base_acc:.2f}%")
print(f"\nТочность с оригинальным слоем ImageNette: {new_acc:.2f}%")


Базовая точность: 97.15%

Точность с оригинальным слоем ImageNette: 3.80%


## Шаг 6: Дообучить последний слой на ImageNette и проверить удалось ли добиться исходного качества.

### 6.1 Заморозка слоев кроме последнего

In [202]:
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

### 6.2 Обучение на ImageNette (последний слой)

In [203]:
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.0001)
last_acc = 0.0

model, last_acc = model_train(model, imagenette_train, imagenette_val, optimizer, save_fc=False)

print(f"Базовая точность: {base_acc:.2f}%")
print(f"Попытка вернуть точность: {last_acc:.2f}%")

Epoch 1:   0%|          | 0/296 [00:00<?, ?it/s]

Epoch 5: 100%|██████████| 296/296 [03:09<00:00,  1.56it/s]


Базовая точность: 97.15%
Попытка вернуть точность: 69.68%


# Вывод:

Текст вывода добавить